<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/END_NoCoDeR_AutoTokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Mar  7 04:10:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import pandas as pd
import numpy as np
import fileinput
import re
import itertools
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
#import seaborn as sns
import dateutil.parser
import datetime
#from ipyfilechooser import FileChooser
import numpy as np
import os
import gzip
import dateutil.parser
from datetime import datetime
import sys
import glob
import matplotlib.dates as mdates
from datetime import timedelta
import ipywidgets as widgets
from IPython.display import display
import torch
import json
import random
import spacy
from pprint import pprint
import six
import sys, token, tokenize
import ast
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
import math
import time
from torch.utils.data.distributed import DistributedSampler
#from tensorboardX import SummaryWriter
from tqdm import tqdm, trange
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 3.2MB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4508c133a96af1bb3fd69d5d1aaee995f6adc2ef0766fcecdf93c97f5a185c6c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!wget -O gpt2_bpe_encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json


In [ ]:
!unzip /content/python.zip
!cp /content/python/final/jsonl/train/python_train_0.jsonl.gz .
!gzip -d /content/python_train_0.jsonl.gz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/english_python_data.txt .
!cp /content/drive/MyDrive/EVA4/END_Capstone/english_python_cleaned.txt .
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone.csv .

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_baseline.pt .

In [ ]:
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from transformers import AutoTokenizer, AutoModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
auto_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
spacy_en = spacy.load('en')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]
#model = RobertaModel.from_pretrained("microsoft/codebert-base")
#model.to(device)

In [ ]:
"""
    Removes docstrings and comments
"""
def remove_docstrings_comments(src_string, doc_string=None, debug=False):
    mod = []

    prev_toktype = token.INDENT
    first_line = None
    last_lineno = -1
    last_col = 0
    try:
        #tokgen = tokenize.generate_tokens(source.readline)
        tokgen = tokenize.generate_tokens(six.StringIO(src_string.rstrip()).readline)
        for toktype, ttext, (slineno, scol), (elineno, ecol), ltext in tokgen:
            if 0:   # Change to if 1 to see the tokens fly by.
                print("%10s %-14s %-20r %r" % (
                    tokenize.tok_name.get(toktype, toktype),
                    "%d.%d-%d.%d" % (slineno, scol, elineno, ecol),
                    ttext, ltext
                    ))
            if slineno > last_lineno:
                last_col = 0
            if scol > last_col:
                mod.append(" " * (scol - last_col))
            if toktype == token.STRING and prev_toktype == token.INDENT:
                # Docstring
                mod.append("#--")
            elif toktype == tokenize.COMMENT:
                # Comment
                mod.append("##")
            else:
                mod.append(ttext)
            prev_toktype = toktype
            last_col = ecol
            last_lineno = elineno
        return "".join(mod)
    except:
        print(doc_string)
        print(src_string )
        print(sys.exc_info())

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Create datasets and dataloaders

In [ ]:
nl_to_pl_df = pd.read_csv('/content/end_capstone.csv')

In [ ]:
nl_to_pl_df

In [ ]:
print(nl_to_pl_df['code_len'].max(),nl_to_pl_df['code_len'].min())
print(nl_to_pl_df['docstring_len'].max(),nl_to_pl_df['docstring_len'].min())
nl_to_pl_df[nl_to_pl_df['code_len'] ==0]

2443 11
313 16


,docstring,code,docstring_len,code_len,cleaned_code,cleaned_code_len


In [ ]:
nl_to_pl_df[(nl_to_pl_df['code_len'] > 256) & (nl_to_pl_df['code_len'] < 512)] 

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    # if n_gpu > 0:
    #     torch.cuda.manual_seed_all(seed)
set_seed(0x1112233)

In [ ]:
tok_ids_list=[]
class NLPLSingleEntry(object):
    """A single set of features of data."""

    def __init__(self, 
                 code_ids, 
                 code_mask, 
                 doc_ids,
                 doc_mask,
                 ):
        self.code_ids = code_ids
        self.code_mask = code_mask
        self.doc_ids = doc_ids
        self.doc_mask = doc_mask
        #self.segment_ids = segment_ids
        #self.label_id = label_id
class NLPLDataSet():
    def __init__(self, 
                 doc_tokenizer, 
                 code_tokenizer):
        self.doc_tokenizer = doc_tokenizer
        self.code_tokenizer = code_tokenizer

    def prepare_tokens(self, 
                       samples, 
                       tokenizer, 
                       max_seq_length=0,
                       data_type=None):
        """
            Tokenizes an input sequence, adds padding and SOS+EOS 
        """
        toks = tokenizer.tokenize(samples)
        # print(data_type)
        # print(toks)
        if max_seq_length > 2 and len(toks) > max_seq_length - 2:
            toks = toks[:max_seq_length -2]
        tok_ids =  tokenizer.convert_tokens_to_ids(toks)
        ### We use pseudo-BERT process so we will add both CLS and SEP tokens for
        ### src and target inputs
        tok_ids = [tokenizer.cls_token_id] + tok_ids + [tokenizer.sep_token_id]
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [ 1 ] * len(tok_ids)

        if len(tok_ids) < max_seq_length:
            padding_length = max_seq_length - len(tok_ids)
            tok_ids = tok_ids + ([tokenizer.pad_token_id] * padding_length)
            input_mask = input_mask + ([ 0 ] * padding_length) ### Padded tokens are zero-masked
        
        # print(tok_ids)
        return tok_ids, input_mask

    def create_dataset(self,
                    nl_to_pl_df,
                    final_ds,
                    sample_count=10000,
                    max_doc_len=50,
                    max_code_len=0):
        """
            Reads from a dataframe, tokenizes and numericalizes both docstrings 
            and code. 

        """
        #final_ds = []
        for idx in nl_to_pl_df.itertuples():
            ## For SOS and EOS tokens 2 positions are left
            if not idx.cleaned_code:
                print("Invalid entry, No code found for:", idx.docstring)
            
            doc_toks, doc_mask = self.prepare_tokens(idx.docstring,
                                                      self.doc_tokenizer,
                                                      max_doc_len,
                                                      "docs")
            code_toks, code_mask = self.prepare_tokens(idx.cleaned_code,
                                                      self.code_tokenizer,
                                                      max_code_len,
                                                      "code")
            #code_toks = None
            ### Skip over current iteration if no valid code found

            # print(code_toks)
            # print(code_mask)
            # print(doc_toks)
            # print(doc_mask)
            final_entry = NLPLSingleEntry(code_toks, 
                                            code_mask, 
                                            doc_toks, 
                                            doc_mask)
            # print(final_entry.code_ids)
            # print(final_entry.code_mask)
            # print(final_entry.doc_ids)
            # print(final_entry.doc_mask)
            final_ds.append(final_entry)
        #print(len(final_ds))
        return final_ds
        

In [ ]:
nl_to_pl_df[(nl_to_pl_df['cleaned_code_len'] >= MAX_VOCAB_LENGTH) & (nl_to_pl_df['cleaned_code_len'] >= MAX_LENGTH)]

In [ ]:
### We will use the same tokenizer for both docstrings and code
final_ds = []
MAX_LENGTH=256
MAX_VOCAB_LENGTH=256
assert(MAX_VOCAB_LENGTH <= MAX_LENGTH)
#selected_elems = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] <= MAX_LENGTH]
#selected_elems = my_df_copy[my_df_copy['code_len'] <= MAX_LENGTH]
selected_elems = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] <= MAX_VOCAB_LENGTH]
#selected_elems = nl_to_pl_df[(nl_to_pl_df['cleaned_code_len'] >= MAX_VOCAB_LENGTH) & (nl_to_pl_df['cleaned_code_len'] = MAX_LENGTH)]
my_nlpl_ds = NLPLDataSet(auto_tokenizer, auto_tokenizer).create_dataset(selected_elems, 
                                                                        final_ds, 
                                                                        max_doc_len=MAX_VOCAB_LENGTH, 
                                                                        max_code_len=MAX_VOCAB_LENGTH)


In [ ]:
all_code_ids = torch.tensor([f.code_ids for f in my_nlpl_ds], dtype=torch.long)
all_code_mask = torch.tensor([f.code_mask for f in my_nlpl_ds], dtype=torch.long)
all_doc_ids = torch.tensor([f.doc_ids for f in my_nlpl_ds], dtype=torch.long)
all_doc_mask = torch.tensor([f.doc_mask for f in my_nlpl_ds], dtype=torch.long)
# if output_mode == "classification":
#     all_label_ids = torch.tensor([f.label_id for f in my_nlpl_ds], dtype=torch.long)

train_dataset = TensorDataset(all_code_ids, 
                              all_code_mask, 
                              all_doc_ids, 
                              all_doc_mask)




In [ ]:
dataset_size = len(train_dataset)
dataset_indices = list(range(dataset_size))
np.random.shuffle(dataset_indices)
val_split_index = int(np.floor(0.2 * dataset_size))

train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

In [ ]:
BATCH_SIZE=16
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)


#train_sampler = RandomSampler(train_dataset,) #if args.local_rank == -1 else DistributedSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE, shuffle=False)
val_dataloader = DataLoader(train_dataset, sampler=val_sampler, batch_size=BATCH_SIZE, shuffle=False)

#train_sampler = RandomSampler(train_dataset) #if args.local_rank == -1 else DistributedSampler(train_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

In [ ]:
iter_one = next(iter(train_dataloader))
iter_one_val = next(iter(val_dataloader))

#iter_one[0]=all_code_ids, 
#iter_one[1]=all_code_mask, 
#iter_one[2]=all_doc_ids, 
#iter_one[3]=all_doc_mask
#mask_reshape.shape
# trg = iter_one[0]
# trg_len = trg.shape[1]
# trg_pad_mask = iter_one[1].unsqueeze(1).unsqueeze(2) 
# trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len))).bool()

# #trg_sub_mask = [trg len, trg len]
    
# trg_mask = trg_pad_mask & trg_sub_mask

### Model Definitions

In [ ]:
class TransEncoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = MAX_LENGTH):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([TransEncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [ ]:
class TransEncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class TransDecoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 1000):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([TransDecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        #output =F.softmax(output, dim=2)  
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [ ]:
class TransDecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
class TransSeq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device,
                 ):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src_mask):
        
        #src = [batch size, src len]
        
        src_mask = src_mask.unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg, trg_mask):
        
        #trg = [batch size, trg len]
        trg_pad_mask = trg_mask.unsqueeze(1).unsqueeze(2) 
        """
            A boolean tensor of shape [batch size, 1, 1, trg len]
        """
        
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, src_mask, trg, trg_mask):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src_mask)
        trg_mask = self.make_trg_mask(trg, trg_mask)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [ ]:
INPUT_DIM = auto_tokenizer.vocab_size
OUTPUT_DIM = auto_tokenizer.vocab_size
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 4
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = TransEncoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = TransDecoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device,
              max_length=MAX_LENGTH)

In [ ]:
INPUT_DIM = auto_tokenizer.vocab_size
OUTPUT_DIM = auto_tokenizer.vocab_size
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 1024
DEC_PF_DIM = 1024
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = TransEncoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = TransDecoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device,
              max_length=MAX_LENGTH)

In [ ]:
!nvidia-smi

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_baseline_128.pt .

In [ ]:
SRC_PAD_IDX = auto_tokenizer.pad_token_id #SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = auto_tokenizer.pad_token_id #TRG.vocab.stoi[TRG.pad_token]

model = TransSeq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 44,314,457 trainable parameters


In [ ]:
def train(model, iterator, optimizer, criterion, clip, device):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        trg = batch[0].to(device)
        trg_mask = batch[1].to(device)
        src = batch[2].to(device)
        src_mask = batch[3].to(device)
        
        
        optimizer.zero_grad()
        
        output, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            trg = batch[0].to(device)
            trg_mask = batch[1].to(device)
            src = batch[2].to(device)
            src_mask = batch[3].to(device)

            output, _ = model(src, src_mask, trg[:,:-1], trg_mask[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
loss = criterion(output, trg)

In [ ]:
loss.item()

10.818306922912598

In [ ]:
import time
import math

def run_train_eval_loop(model, 
                        train_dataloader,
                        val_dataloader,
                        optimizer,
                        criterion,
                        device,
                        epochs=20,
                        clip=1,
                        best_valid_loss=float('inf'),
                        file_path='end_capstone_baseline_128.pt',
                        scheduler=None):
    
    for epoch in range(epochs):
    
        start_time = time.time()
        
        train_loss = train(model, train_dataloader, optimizer, criterion, clip, device)
        valid_loss = evaluate(model, val_dataloader, criterion, device)
        if(scheduler is not None):
            scheduler.step(valid_loss)

        end_time = time.time()
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save({"model":model.state_dict(),
                "optimizer":optimizer.state_dict(),
                "loss":valid_loss,
                },file_path)
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
torch.save({"model":model.state_dict(),
    "optimizer":optimizer.state_dict(),
    "loss":7.88,
    },'enc_mb_nowls_1024.pt')

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
file_path='end_capstone_128sq_sth.pt'
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, patience=5, min_lr=1e-8,verbose=True)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=100,
                    clip=1,
                    best_valid_loss=float('inf'),
                    file_path=file_path,
                    scheduler=scheduler)

Epoch: 01 | Time: 0m 14s
	Train Loss: 5.885 | Train PPL: 359.771
	 Val. Loss: 4.147 |  Val. PPL:  63.233
Epoch: 02 | Time: 0m 14s
	Train Loss: 3.572 | Train PPL:  35.599
	 Val. Loss: 3.289 |  Val. PPL:  26.810
Epoch: 03 | Time: 0m 14s
	Train Loss: 2.956 | Train PPL:  19.221
	 Val. Loss: 2.900 |  Val. PPL:  18.171
Epoch: 04 | Time: 0m 14s
	Train Loss: 2.583 | Train PPL:  13.236
	 Val. Loss: 2.643 |  Val. PPL:  14.059
Epoch: 05 | Time: 0m 14s
	Train Loss: 2.308 | Train PPL:  10.056
	 Val. Loss: 2.417 |  Val. PPL:  11.211
Epoch: 06 | Time: 0m 14s
	Train Loss: 2.071 | Train PPL:   7.933
	 Val. Loss: 2.255 |  Val. PPL:   9.537
Epoch: 07 | Time: 0m 14s
	Train Loss: 1.886 | Train PPL:   6.593
	 Val. Loss: 2.128 |  Val. PPL:   8.399
Epoch: 08 | Time: 0m 14s
	Train Loss: 1.710 | Train PPL:   5.530
	 Val. Loss: 2.005 |  Val. PPL:   7.430
Epoch: 09 | Time: 0m 14s
	Train Loss: 1.565 | Train PPL:   4.782
	 Val. Loss: 1.948 |  Val. PPL:   7.014
Epoch: 10 | Time: 0m 14s
	Train Loss: 1.436 | Train PPL

In [ ]:
file_path='end_capstone_128sq_sth.pt'

chkpt = torch.load(file_path)
print(chkpt['loss'])

1.5051638835242815


In [ ]:
model.load_state_dict(chkpt['model'])

<All keys matched successfully>

In [ ]:
file_path='end_capstone_128sq_sth256_2.pt'
LEARNING_RATE = 0.0005

#optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, patience=5, min_lr=1e-8,verbose=True)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=100,
                    clip=1,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path,
                    scheduler=scheduler)

Epoch: 01 | Time: 0m 50s
	Train Loss: 2.898 | Train PPL:  18.143
	 Val. Loss: 2.943 |  Val. PPL:  18.965
Epoch: 02 | Time: 0m 52s
	Train Loss: 2.889 | Train PPL:  17.974
	 Val. Loss: 2.945 |  Val. PPL:  19.015
Epoch: 03 | Time: 0m 51s
	Train Loss: 2.882 | Train PPL:  17.858
	 Val. Loss: 2.954 |  Val. PPL:  19.177
Epoch: 04 | Time: 0m 51s
	Train Loss: 2.889 | Train PPL:  17.984
	 Val. Loss: 2.963 |  Val. PPL:  19.359
Epoch: 05 | Time: 0m 51s
	Train Loss: 2.893 | Train PPL:  18.043
	 Val. Loss: 2.973 |  Val. PPL:  19.553
Epoch: 06 | Time: 0m 51s
	Train Loss: 2.888 | Train PPL:  17.957
	 Val. Loss: 2.938 |  Val. PPL:  18.883
Epoch: 07 | Time: 0m 51s
	Train Loss: 2.894 | Train PPL:  18.061
	 Val. Loss: 2.972 |  Val. PPL:  19.527
Epoch: 08 | Time: 0m 51s
	Train Loss: 2.887 | Train PPL:  17.931
	 Val. Loss: 2.957 |  Val. PPL:  19.240
Epoch: 09 | Time: 0m 51s
	Train Loss: 2.893 | Train PPL:  18.056
	 Val. Loss: 2.929 |  Val. PPL:  18.709
Epoch: 10 | Time: 0m 51s
	Train Loss: 2.872 | Train PPL

In [ ]:
LEARNING_RATE = 0.00005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=20,
                    clip=1.4,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path)

In [ ]:
chkpt['loss']

1.520801103340005

In [ ]:
file_path='end_capstone_codesearch_256_seqLen512.pt'
LEARNING_RATE = 0.00005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=20,
                    clip=1.4,
                    best_valid_loss=5.050,
                    file_path=file_path)

In [ ]:
run_train_eval_loop(model,
                    train_dataloader,
                    val_dataloader,
                    optimizer,
                    criterion,
                    device,
                    epochs=20,
                    clip=1,
                    best_valid_loss=chkpt['loss'],
                    file_path=file_path)

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_256_mod2.pt /content/

In [ ]:
!cp /content/end_capstone_LRManaged_256.pt /content/drive/MyDrive/EVA4/END_Capstone/

In [ ]:
chkpt = torch.load("/content/end_capstone_128sq_sth.pt")
model.load_state_dict(chkpt['model'])


<All keys matched successfully>

In [ ]:
!cp tut6-model.pt /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_256_mod2.pt

In [ ]:
sentence="Write a python program to add 2 numbers"
tokenizer = auto_tokenizer
dataset_handler = NLPLDataSet(tokenizer, tokenizer)
src_indexes, src_mask =  dataset_handler.prepare_tokens(sentence, dataset_handler.doc_tokenizer)

In [ ]:
src_mask = torch.LongTensor(src_mask).unsqueeze(0)#.to(device)
    
src_mask = model.make_src_mask(src_mask)

In [ ]:
!cp /content/end_capstone_codesearch_256_seqLen512.pt /content/drive/MyDrive/EVA4/END_Capstone

In [ ]:
!cp /content/drive/MyDrive/EVA4/END_Capstone/end_capstone_codesearch_256_seqLen512.pt .

In [ ]:
def get_code(sentence,
             tokenizer,
             model, 
             device, 
             max_len = 100):
    
    model.eval()
    dataset_handler = NLPLDataSet(tokenizer, tokenizer)
    src_indexes, src_mask =  dataset_handler.prepare_tokens(sentence, dataset_handler.doc_tokenizer)
    # if isinstance(sentence, str):
    #     nlp = spacy.load('de')
    #     tokens = [token.text.lower() for token in nlp(sentence)]
    # else:
    #     tokens = [token.lower() for token in sentence]
    #dataset_handler.prepare_tokens()

    #tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
        
    #src_indexes = [tokenizer.convert_tokens_to_ids[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = torch.LongTensor(src_mask).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_mask)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [tokenizer.cls_token_id]
    #trg_mask = [1]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        base_mask = torch.LongTensor([1]*(i+1)).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor, base_mask)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == tokenizer.sep_token_id:
            break
    
    trg_tokens = [tokenizer.convert_ids_to_tokens(i) for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
model.load_state_dict(chkpt['model'])

In [ ]:
input_text = "Write a python program to check if a string has digits"
splitted_text = auto_tokenizer.tokenize(input_text)
mycode, attention_val = get_code(input_text,auto_tokenizer, model, device,max_len=200)
output = auto_tokenizer.convert_tokens_to_string(mycode[:-1])
print(output)

if(isinstance(ch) == 1):
    print(ord(ch)


In [ ]:
attention_val[:,1:2,:,:].shape

torch.Size([1, 1, 135, 18])

In [ ]:
import matplotlib.ticker as ticker

display_attention(splitted_text, mycode, attention_val[:,1,:,:].unsqueeze(1), n_heads=1, n_rows=1, n_cols=1)
#display_attention(splitted_text, mycode, attention_val[:,4:8,:,:], n_heads=4, n_rows=2, n_cols=2)

In [ ]:
[t.lower() for t in input_text]

In [ ]:
def right_posth_posth(pos):
    global left_pos
    while(pos >= 0):
           if right += 1:
                 break
          return 'Not a palindrome'

n = 'C'
if right_pos >= left rotation:

In [ ]:
selected_strings = nl_to_pl_df[nl_to_pl_df['cleaned_code_len'] > 128]['docstring'].values
all_strings = selected_strings
offset=2000
for one_string in all_strings[offset:offset+50]:
    cleaned_string = one_string.rstrip('\n').lstrip('#')
    mycode, attention_val = get_code(cleaned_string,auto_tokenizer, model, device, max_len=512)
    print(cleaned_string)
    print(auto_tokenizer.convert_tokens_to_string(mycode[:-1]))


 Remove special symbols/Punctuation from a given string
import re
regex = '\.[0]*'
def remove_leading_zeros(ip):
    modified_ip = re.sub(regex, '.', ip)
    return modified_ip
 From given string replace each punctuation with #
sentence = 'the quick brown fox'
words = sentence.split(' ')
words.reverse()
print(' '.join(words))
 Given a list iterate it and count the occurrence of each element and create a dictionary to show the count of each elemen

list1 = ['apple', 'orange', 'banana']
list2 = [1, 2, 3]
print(list_list_list_to_list2))
 Given a two list of equal size create a set such that it shows the element from both lists in the pair
list1 = [1, 2, 3, 4, 5]
list2 = [5, 4, 3, 2, 2, 3, 4, 2, 3, 4]
diff_list = [a-b for (a,b) in zip(list1, list2)]
print(diff_list)
 Given a two sets find the intersection and remove those elements from the first set

set1 = {10, 20, 30, 40, 50}
set2 = {30, 40, 50, 60, 70}

print(set1.symmetric_difference(set2))
 Given a dictionary get all values from the d